Application of Albumentations library for data augmentation with pytorch torchvision library

In [5]:
import albumentations as alb
import os,random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torchvision import transforms,datasets
from albumentations.pytorch import ToTensorV2

In [4]:
# basic torch visison transformer
torchvision_transform = transforms.Compose(
    [
        transforms.Resize((32, 32)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

In [6]:
train1 = datasets.CIFAR10(
    root="./data", train=True, download=True, transform=torchvision_transform
)
test1 = datasets.CIFAR10(
    root="./data", train=False, download=True, transform=torchvision_transform
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 170498071/170498071 [00:01<00:00, 104509967.72it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# If you are using normal datasets---

```python
class TorchvisionDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        label = self.labels[idx]
        file_path = self.file_paths[idx]

        # Read an image with PIL
        image = Image.open(file_path)

        # application of transforms
        if self.transform:
            image = self.transform(image)
        return image, label


torchvision_transform = transforms.Compose(
    [
        transforms.Resize((256, 256)),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        ),
    ]
)


torchvision_dataset = TorchvisionDataset(
    file_paths=["./images/image_1.jpg", "./images/image_2.jpg", "./images/image_3.jpg"],
    labels=[1, 2, 3],
    transform=torchvision_transform,
)
```

Application of albumentations instead of torch vision transformers

In [12]:
alb_transformation = alb.Compose(
    [
        alb.Resize(32, 32),
        alb.HorizontalFlip(),
        alb.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010)),
        ToTensorV2(),
    ]
)

In [13]:
train2 = datasets.CIFAR10(
    root="./data", train=True, download=True, transform=alb_transformation
)
test2 = datasets.CIFAR10(
    root="./data", train=False, download=True, transform=alb_transformation
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 170498071/170498071 [00:01<00:00, 103895396.48it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [19]:
print(train2.data.shape)
print(len(train2.targets))
print(type(train2.data))
print(np.mean(train2.data))
print(np.std(train2.data))
print(np.sum(train2.data))

(50000, 32, 32, 3)
50000
<class 'numpy.ndarray'>
120.70756512369792


64.1500758911213
18540682003


In [20]:
print(train1.data.shape)
print(len(train1.targets))
print(type(train1.data))
print(np.mean(train1.data))
print(np.std(train1.data))
print(np.sum(train1.data))

(50000, 32, 32, 3)
50000
<class 'numpy.ndarray'>
120.70756512369792
64.1500758911213
18540682003


##### hence it proved that we can use albumenation transforms inplace of torchvision transform 

```python
class AlbumentationsPilDataset(Dataset):
    """__init__ and __len__ functions are the same as in TorchvisionDataset"""
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        label = self.labels[idx]
        file_path = self.file_paths[idx]

        image = Image.open(file_path)

        if self.transform:
            # Convert PIL image to numpy array
            image_np = np.array(image)
            # Apply transformations
            augmented = self.transform(image=image_np)
            # Convert numpy array to PIL Image
            image = Image.fromarray(augmented['image'])
        return image, label


albumentations_pil_transform = A.Compose([
    A.Resize(256, 256),
    A.RandomCrop(224, 224),
    A.HorizontalFlip(),
])


# Note that this dataset will output PIL images and not numpy arrays nor PyTorch tensors
albumentations_pil_dataset = AlbumentationsPilDataset(
    file_paths=['./images/image_1.jpg', './images/image_2.jpg', './images/image_3.jpg'],
    labels=[1, 2, 3],
    transform=albumentations_pil_transform,
)
```

```
Albumentations equivalents for torchvision transforms::

torchvision transform	||Albumentations transform||	Albumentations example
Compose	||Compose	||A.Compose([A.Resize(256, 256), A.RandomCrop(224, 224)])
CenterCrop	||CenterCrop||	A.CenterCrop(256, 256)
ColorJitter	||HueSaturationValue||	A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5)
Pad	||PadIfNeeded||	A.PadIfNeeded(min_height=512, min_width=512)
RandomAffine||	Affine||	A.Affine(scale=(0.9, 1.1), translate_percent=(0.0, 0.2), rotate=(-45, 45), shear=(-15, 15), mode=cv2.BORDER_REFLECT_101, p=0.5)
RandomCrop||	RandomCrop||	A.RandomCrop(256, 256)
RandomGrayscale||	ToGray||	A.ToGray(p=0.5)
RandomHorizontalFlip||	HorizontalFlip||	A.HorizontalFlip(p=0.5)
RandomPerspective	||Perspective	||A.Perspective(scale=(0.2, 0.4), fit_output=True, p=0.5)
RandomRotation||	Rotate||	A.Rotate(limit=45, p=0.5)
RandomVerticalFlip||	VerticalFlip||	A.VerticalFlip(p=0.5)
Resize||	Resize||	A.Resize(256, 256)
GaussianBlur||	GaussianBlur||	A.GaussianBlur(blur_limit=(3, 7), p=0.5)
RandomInvert	||InvertImg||	A.InvertImg(p=0.5)
RandomPosterize||	Posterize||	A.Posterize(num_bits=4, p=0.5)
RandomSolarize||	Solarize||	A.Solarize(threshold=127, p=0.5)
RandomAdjustSharpness||	Sharpen	||A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.5)
RandomAutocontrast	||RandomBrightnessContrast||	A.RandomBrightnessContrast(brightness_limit=0, contrast_limit=0.2, p=0.5)
RandomEqualize||	Equalize||	A.Equalize(p=0.5)
RandomErasing||	CoarseDropout||	A.CoarseDropout(min_height=8, max_height=32, min_width=8, max_width=32, p=0.5)
Normalize||	Normalize||	A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
```

Very Imp References:

https://albumentations.ai/docs/examples/pytorch_classification/

https://towardsdatascience.com/getting-started-with-albumentation-winning-deep-learning-image-augmentation-technique-in-pytorch-47aaba0ee3f8

https://debuggercafe.com/image-augmentation-using-pytorch-and-albumentations/

https://www.youtube.com/watch?v=rAdLwKJBvPM